## Basic Functionality

Regression tables are generated with `ETable()`, which can output different formats such as html, docx, latex, for instance using the [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) package or generating formatted LaTex Tables using [booktabs](https://ctan.org/pkg/booktabs?lang=en). 

In [38]:
import numpy as np
import pandas as pd
import pyfixest as pf
import tabout as to
import statsmodels.formula.api as smf
data = pf.get_data()

In [39]:
sfit1=smf.ols("Y ~ X1", data=data).fit()
sfit2=smf.ols("Y ~ X1+X2", data=data).fit()
to.ETable([sfit1, sfit2])

In [3]:
ffit1=pf.feols("Y ~ X1", data=data)
ffit2=pf.feols("Y ~ X1+X2", data=data)
to.ETable([sfit1,sfit2,ffit1, ffit2])

In [4]:
fit1 = pf.feols("Y ~ X1 + X2 | f1", data=data)
fit2 = pf.feols("Y ~ X1 + X2 | f1 + f2", data=data)
fit3 = pf.feols("Y ~ X1 *X2 | f1 + f2", data=data)
fit4 = pf.feols("Y2 ~ X1 + X2 | f1", data=data)
fit5 = pf.feols("Y2 ~ X1 + X2 | f1 + f2", data=data)
fit6 = pf.feols("Y2 ~ X1 *X2 | f1 + f2", data=data)

# Regression Tables via `ETable()`

## Basic Usage

We can compare all regression models via `ETable()` function:


In [5]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6])

You can also estimate and display multiple regressions with one line of code using the (py)fixest stepwise notation:

In [6]:
to.ETable(pf.feols("Y+Y2~csw(X1,X2,X1:X2)", data=data))

## Keep and drop variables
For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match.


In [7]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], keep="X1")

We can use the `exact_match` argument to select a specific set of variables:

In [8]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], keep=["X1", "X2"], exact_match=True)

We can also easily **drop** variables via the `drop` argument:

In [9]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], drop=["X1"])

## Specify displayed model statistics
The user can specify the model statistics to be displayed and their order by passing a list of strings to `model_stats`. Names of the statistics must match the model's respective attribute names such as "r2", "adj_r2", "N", "r2_within" (see the respective function reference for attributes and omit the leading "_"). The type of standard error estimated can be shown by adding "se_type" to the list. When passing an empty list no model statistics are displayed.

In [10]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], model_stats=['N','r2','r2_within'])

Model statistics can also be relabeled by passing a dictionary `model_stats_labels`.

In [11]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], model_stats=['N','r2'], model_stats_labels={"N": "Number of firms", "r2": "R squared"})

## Hide fixed effects
We can hide the rows showing the relevant fixed effects by setting `show_fe=False` (for instance when the set of fixed effects is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table).

In [12]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], model_stats=['r2','N'], show_fe=False)

## Display p-values or confidence intervals
By default, `to.ETable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument.


In [13]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], coef_fmt="b \n (se) \n [p]")

## Significance levels and rounding
Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the `signif_code` and `digits` function arguments:


In [14]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits=5)

## Output tables to different formats
You can save the table as **docx**, **html**, or **latex** using TabOut's `save()` method and use the `file_name=` argument to specify the respective path where it should be saved.. For example, to save the table as a Word document:

In [15]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits=5).save(type="docx", file_name="docs/regression_table.docx",replace=True)

GT(_tbl_data=  level_0       level_1                           0  \
0    coef            X1  -0.94953*** <br> (0.06656)   
1    coef            X2  -0.17423*** <br> (0.01841)   
2    coef       X1 x X2                               
3      fe            f1                           x   
4      fe            f2                           -   
5   stats  Observations                         997   
6   stats            R2                     0.48899   

                            1                           2  \
0  -0.92405*** <br> (0.06187)  -0.92417*** <br> (0.06188)   
1  -0.17411*** <br> (0.01483)  -0.18550*** <br> (0.02555)   
2                                  0.01057 <br> (0.01846)   
3                           x                           x   
4                           x                           x   
5                         997                         997   
6                     0.65904                     0.65916   

                            3                           4  \
0  -1.26655*** <br> (0.17368)  -1.23153*** <br> (0.19524)   
1  -0.13056*** <br> (0.04241)  -0.11767*** <br> (0.04216)   
2                                                           
3                           x                           x   
4                           -                           x   
5                         998                         998   
6                     0.12017                     0.17151   

                            5  
0  -1.23100*** <br> (0.19462)  
1     -0.07369 <br> (0.10515)  
2     -0.04082 <br> (0.08218)  
3                           x  
4                           x  
5                         998  
6                     0.17180  , _body=<great_tables._gt_data.Body object at 0x000002A8693C6F00>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002A8662145F0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.1, ** p < 0.05, *** p < 0.01. Format of coefficient cell: Coefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002A8662156D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', t

To obtain latex output use `format = "tex"`.  Etable will use latex packages `booktabs`, `threeparttable` and `makecell` for the table layout, so don't forget to include these packages in your latex document.

## Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).


In [16]:
labels = {
    "Y": "Wage",
    "Y2": "Wealth",
    "X1": "Age",
    "X2": "Years of Schooling",
    "f1": "Industry",
    "f2": "Year",
}

to.ETable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels)

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.


In [17]:
to.ETable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
)

## Setting defaults
You can set default options for `ETable` by changing the class attributes. For example, to change the default model statistics, always use the same labels, digits, and significance levels, you can do the following:

In [18]:
to.ETable.DEFAULT_MODEL_STATS = ["N","r2"]
to.ETable.DEFAULT_LABELS = labels
to.ETable.DEFAULT_DIGITS = 2
to.ETable.DEFAULT_SIGNIF_CODE = [0.01, 0.05, 0.1]


In [19]:
to.ETable([fit1, fit2, fit3, fit4, fit5, fit6])

## Rename categorical variables

By default, categorical variables are returned using the formulaic "C(variable)[T.value]" notation. Via the `cat_template` argument,
you can rename categorical variables via a specified template *{variable}={value}*. This works when either the variable is categorial in the DataFrame, or the C() or i() operators are used in the regresson formula.
´

In [20]:
# Add a categorical variable
data['job'] = np.random.choice(["Managerial", "Admin", "Blue collar"], size=len(data), p=[1/3, 1/3, 1/3])
# Add a label for this variable to the dictionary
labels['job']="Job Family"

fit7 = pf.feols("Y ~ X1 + X2 + job", data = data)

to.ETable([fit7], labels=labels, cat_template = "{variable}: {value}")

But you can also remove the variable name and only keep the levels (categories) by specifying *cat_template="{value}"*. Note that the labeling of categories also works in interaction terms:

In [21]:
fit7 = pf.feols("Y ~ X1 + X2 + job", data = data)
fit8 = pf.feols("Y ~ X1 + X2 + job*X2", data = data)

to.ETable([fit7, fit8], labels=labels, cat_template="{value}")

## Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.


In [22]:
to.ETable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    model_heads=["US", "China", "EU", "US", "China", "EU"],
)

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.


In [23]:
to.ETable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
)

Remove the dependent variables from the headers:

In [24]:
to.ETable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="",
)

## Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.


In [25]:
to.ETable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
        "Additional Info": ["A", "A", "B", "B", "C", "C"],
    },
)

## Custom table notes
You can replace the default table notes with your own notes using the `notes` argument.

In [26]:
mynotes = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
to.ETable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    notes=mynotes,
)

## Publication-ready LaTex tables
With few lines of code you thus obtain a publication-ready latex table:


In [28]:
from IPython.display import FileLink
import pylatex as pl 

# Use pylatex to create a tex file with the table
def make_pdf(tab, file):
    "Create a PDF document with tex table."
    doc = pl.Document()
    doc.packages.append(pl.Package("booktabs"))
    doc.packages.append(pl.Package("threeparttable"))
    doc.packages.append(pl.Package("makecell"))

    with (
        doc.create(pl.Section("A PyFixest LateX Table")),
        doc.create(pl.Table(position="htbp")) as table,
    ):
        table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)



tab = to.ETable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    notes=mynotes,
    show_fe=True,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
    },
).make(type="tex")
    

# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "docs/SampleTableDoc2")
display(FileLink("docs/SampleTableDoc2.pdf"))

c:\PythonProjects\tabout\docs\SampleTableDoc2.pdf


# Descriptive Statistics via `to.DTable()`

The function `to.DTable()` allows to display descriptive statistics for a set of variables in the same layout.

## Basic Usage of dtable
Specify the variables you want to display the descriptive statistics for. You can also use a dictionary to rename the variables and add a caption.


In [29]:
to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    caption="Descriptive statistics",
    digits=2,
)

Choose the set of statistics to be displayed with `stats`. You can use any pandas aggregation functions.


In [30]:
to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    stats=["count", "mean", "std", "min", "max"],
    labels=labels,
    caption="Descriptive statistics",
)

## Summarize by characteristics in columns and rows
You can summarize by characteristics using the `bycol` argument when groups are to be displayed in columns. When the number of observations is the same for all variables in a group, you can also opt to display the number of observations only once for each group byin a separate line at the bottom of the table with `counts_row_below==True`.


In [31]:
# Generate some categorial data
data["country"] = np.random.choice(["US", "EU"], data.shape[0])
data["occupation"] = np.random.choice(["Blue collar", "White collar"], data.shape[0])

# Drop nan values to have balanced data
data.dropna(inplace=True)

to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country", "occupation"],
    stats=["count", "mean", "std"],
    caption="Descriptive statistics",
    stats_labels={"count": "Number of observations"},
    counts_row_below=True,
)

You can also use custom aggregation functions to compute further statistics or affect how statistics are presented. Pyfixest provides two such functions `mean_std` and `mean_newline_std` which compute the mean and standard deviation and display both the same cell (either with line break between them or not). This allows to have more compact tables when you want to show statistics for many characteristcs in the columns.

You can also hide the display of the statistics labels in the header with `hide_stats_labels=True`. In that case a table note will be added naming the statistics displayed using its label (if you have not provided a custom note).


In [32]:
to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country", "occupation"],
    stats=["mean_newline_std", "count"],
    caption="Descriptive statistics",
    stats_labels={"count": "Number of observations"},
    counts_row_below=True,
    hide_stats=True,
)

You can also split by characteristics in both columns and rows. Note that you can only use one grouping variable in rows, but several in columns (as shown above).


In [33]:
to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country"],
    byrow="occupation",
    stats=["count", "mean", "std"],
    caption="Descriptive statistics",
)

And you can again export descriptive statistics tables also to LaTex:


In [34]:
dtab = to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country"],
    byrow="occupation",
    stats=["count", "mean", "std"],
).make(type="tex")

make_pdf(dtab, "docs/SampleTableDoc3")
display(FileLink("docs/SampleTableDoc3.pdf"))

c:\PythonProjects\tabout\docs\SampleTableDoc3.pdf

# Table Output

`to.ETable()` and `to.DTable()` build on the class `MTable()` which you can use directly to generate tables in the same layout from other pandas dataframes.

## Basic Usage of TabOut


In [ ]:
df = pd.DataFrame(np.random.randn(4, 4).round(2), columns=["A", "B", "C", "D"])

# Make Booktabs style table
to.MTable(df=df, caption="This is a caption", notes="These are notes")

## Mutiindex DataFrames
When the respective dataframe has a mutiindex for the columns, columns spanners are generated from the index. The row index can also be a multiindex (of at most two levels). In this case the first index level is used to generate group rows (for instance using the index name as headline and separating the groups by a horizontal line) and the second index level is used to generate the row labels.


In [ ]:
# Create a multiindex dataframe with random data
row_index = pd.MultiIndex.from_tuples(
    [
        ("Group 1", "Variable 1"),
        ("Group 1", "Variable 2"),
        ("Group 1", "Variable 3"),
        ("Group 2", "Variable 4"),
        ("Group 2", "Variable 5"),
        ("Group 3", "Variable 6"),
    ]
)

col_index = pd.MultiIndex.from_product([["A", "B"], ["X", "Y"], ["High", "Low"]])
df = pd.DataFrame(np.random.randn(6, 8).round(3), index=row_index, columns=col_index)

to.MTable(df=df, caption="This is a caption", notes="These are notes")

You can also hide column group names: This just creates a table where variables on the second level of the row index are displayed in groups based on the first level separated by horizontal lines.


In [37]:
to.TabOut(
    df=df, caption="This is a caption", notes="These are notes", rgroup_display=False
)

AttributeError: module 'tabout' has no attribute 'TabOut'

# Custom Styling with Great Tables
You can use the rich set of methods offered by [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) to further customize the table display when the type is "gt".

## Example Styling

In [ ]:
t=to.ETable([fit1, fit2, fit3, fit4, fit5, fit6]).make(type="gt")
    
t.tab_options(
        column_labels_background_color="cornsilk",
        stub_background_color="whitesmoke",
    )

GT(_tbl_data=  level_0       level_1                       0                       1  \
0    coef            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.062)   
1    coef            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef         X1:X2                                                   
3      fe            f1                       x                       x   
4      fe            f2                       -                       x   
5   stats     S.E. type                  by: f1                  by: f1   
6   stats  Observations                     997                     997   
7   stats            R2                   0.489                   0.659   
8   stats       Adj. R2                       -                       -   
9   stats     R2 Within                   0.239                   0.303   

                        2                       3                       4  \
0  -0.924*** <br> (0.062)  -1.267*** <br> (0.174)  -1.232*** <br> (0.195)   
1  -0.185*** <br> (0.026)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                  by: f1                  by: f1                  by: f1   
6                     997                     998                     998   
7                   0.659                   0.120                   0.172   
8                       -                       -                       -   
9                   0.303                   0.041                   0.039   

                        5  
0  -1.231*** <br> (0.195)  
1     -0.074 <br> (0.105)  
2     -0.041 <br> (0.082)  
3                       x  
4                       x  
5                  by: f1  
6                     998  
7                   0.172  
8                       -  
9                   0.039  , _body=<great_tables._gt_data.Body object at 0x000001223DE580B0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001223D6095E0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell: Coefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001223D60A1B0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layou

## Defining Table Styles: Some Examples

You can easily define table styles that you can apply to all tables in your project. Just define a dictionary with the respective values for the tab options (see the [Great Tables documentation](https://posit-dev.github.io/great-tables/reference/GT.tab_options.html#great_tables.GT.tab_options)) and use the style with `.tab_options(**style_dict)`.


In [ ]:
style_print = {
    "table_font_size": "12px",
    "heading_title_font_size": "12px",
    "source_notes_font_size": "8px",
    "data_row_padding": "3px",
    "column_labels_padding": "3px",
    "row_group_border_top_style": "hidden",
    "table_body_border_top_style": "None",
    "table_body_border_bottom_width": "1px",
    "column_labels_border_top_width": "1px",
    "table_width": "14cm",
}


style_presentation = {
    "table_font_size": "16px",
    "table_font_color_light": "white",
    "table_body_border_top_style": "hidden",
    "table_body_border_bottom_style": "hidden",
    "heading_title_font_size": "18px",
    "source_notes_font_size": "12px",
    "data_row_padding": "3px",
    "column_labels_padding": "6px",
    "column_labels_background_color": "midnightblue",
    "stub_background_color": "whitesmoke",
    "row_group_background_color": "whitesmoke",
    "table_background_color": "whitesmoke",
    "heading_background_color": "white",
    "source_notes_background_color": "white",
    "column_labels_border_bottom_color": "white",
    "column_labels_font_weight": "bold",
    "row_group_font_weight": "bold",
    "table_width": "18cm",
}

In [ ]:
t1 = to.DTable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    stats=["count", "mean", "std", "min", "max"],
    labels=labels,
    caption="Descriptive statistics",
).make(type="gt")

t2 = to.ETable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    show_se_type=False,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
    caption="Regression results",
).make(type="gt")

In [ ]:
display(t1.tab_options(**style_print))
display(t2.tab_options(**style_print))

GT(_tbl_data=                index    N   Mean Std. Dev.     Min    Max
0                Wage  997  -0.13      2.31   -6.54   6.91
1              Wealth  997  -0.32      5.59  -16.97  17.16
2                 Age  997   1.04      0.81    0.00   2.00
3  Years of Schooling  997  -0.13      3.05   -9.67  10.99, _body=<great_tables._gt_data.Body object at 0x000001223E368BF0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='Min', type=<ColInfoTypeEnum.default: 1>, column_label='Min', column_align='center', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001223E41B110>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[''], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001223E41A1B0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='14cm'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='12px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=Opt

GT(_tbl_data=  level_0                   level_1                       0  \
0    coef                       Age  -0.950*** <br> (0.067)   
1    coef        Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age x Years of Schooling                           
3      fe    Industry Fixed Effects                       x   
4      fe        Year Fixed Effects                       -   
5   stats              Observations                     997   
6   stats                        R2                   0.489   
7   stats                   Adj. R2                       -   
8   stats                 R2 Within                   0.239   

                        1                       2                       3  \
0  -0.924*** <br> (0.062)  -0.924*** <br> (0.062)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.026)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                   0.659                   0.659                   0.120   
7                       -                       -                       -   
8                   0.303                   0.303                   0.041   

                        4                       5  
0  -1.232*** <br> (0.195)  -1.231*** <br> (0.195)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.105)  
2                             -0.041 <br> (0.082)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                   0.172                   0.172  
7                       -                       -  
8                   0.039                   0.039  , _body=<great_tables._gt_data.Body object at 0x000001223E419880>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001223E41A930>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title='Regression results', subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell: Coefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001223E41B7A0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='14cm'), table_layout=OptionsInfo(scss=True, categ

In [ ]:
style_printDouble = {
    "table_font_size": "12px",
    "heading_title_font_size": "12px",
    "source_notes_font_size": "8px",
    "data_row_padding": "3px",
    "column_labels_padding": "3px",
    "table_body_border_bottom_style": "double",
    "column_labels_border_top_style": "double",
    "column_labels_border_bottom_width": "0.5px",
    "row_group_border_top_style": "hidden",
    "table_body_border_top_style": "None",
    "table_width": "14cm",
}
display(t1.tab_options(**style_printDouble))
display(t2.tab_options(**style_printDouble))

GT(_tbl_data=                index    N   Mean Std. Dev.     Min    Max
0                Wage  997  -0.13      2.31   -6.54   6.91
1              Wealth  997  -0.32      5.59  -16.97  17.16
2                 Age  997   1.04      0.81    0.00   2.00
3  Years of Schooling  997  -0.13      3.05   -9.67  10.99, _body=<great_tables._gt_data.Body object at 0x000001223E368BF0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='Min', type=<ColInfoTypeEnum.default: 1>, column_label='Min', column_align='center', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001223E41B110>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[''], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001223E41A1B0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='14cm'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='12px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=Opt

GT(_tbl_data=  level_0                   level_1                       0  \
0    coef                       Age  -0.950*** <br> (0.067)   
1    coef        Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age x Years of Schooling                           
3      fe    Industry Fixed Effects                       x   
4      fe        Year Fixed Effects                       -   
5   stats              Observations                     997   
6   stats                        R2                   0.489   
7   stats                   Adj. R2                       -   
8   stats                 R2 Within                   0.239   

                        1                       2                       3  \
0  -0.924*** <br> (0.062)  -0.924*** <br> (0.062)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.026)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                   0.659                   0.659                   0.120   
7                       -                       -                       -   
8                   0.303                   0.303                   0.041   

                        4                       5  
0  -1.232*** <br> (0.195)  -1.231*** <br> (0.195)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.105)  
2                             -0.041 <br> (0.082)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                   0.172                   0.172  
7                       -                       -  
8                   0.039                   0.039  , _body=<great_tables._gt_data.Body object at 0x000001223E419880>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001223E41A930>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title='Regression results', subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell: Coefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001223E41B7A0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='14cm'), table_layout=OptionsInfo(scss=True, categ